In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np

In [2]:
household_path = "my_stata_file.csv"

In [3]:
household_data = pd.read_csv(household_path)

In [4]:
household_data.head()

,Unnamed: 0,hhid,year,region,work10t12,work13t15,work16t18,uct,femaleh,satth,...,m19t60,f61,m61,quint,rural,treatement,pretreatment,posttreatment,treated,ptreat
0,0,1.201060e+11,2005,Sumatra,0.0,0.0,0.0,0,Male,No education completed,...,0.000000,0.166667,0.166667,Quintile 1 (poorest),Rural,1.0,1.0,0.0,1.0,0.548135
1,1,1.201060e+11,2006,Sumatra,0.0,0.0,0.0,1,Male,Primary,...,0.333333,0.000000,0.000000,Quintile 1 (poorest),Rural,1.0,0.0,1.0,1.0,0.490203
2,2,1.201060e+11,2005,Sumatra,0.0,0.0,0.0,0,Male,No education completed,...,0.166667,0.166667,0.166667,Quintile 1 (poorest),Rural,1.0,1.0,0.0,1.0,0.485999
3,3,1.201060e+11,2006,Sumatra,0.0,0.0,0.0,1,Male,No education completed,...,0.166667,0.333333,0.166667,Quintile 1 (poorest),Rural,1.0,0.0,1.0,1.0,0.507705
4,4,1.201060e+11,2005,Sumatra,0.0,0.0,1.0,0,Female,No education completed,...,0.125000,0.000000,0.000000,Quintile 1 (poorest),Rural,1.0,1.0,0.0,1.0,0.605794


In [5]:
del household_data['Unnamed: 0']

In [6]:
#Brief exploration of the data

In [7]:
#hhid refers to the unique, anonymised household id that was created for the study to track household across 2005 and 2006
#region refers to area within Indonesia
#work10t12, work13t15, work16t18 refers to the proportion of household (from 0 to 1) that are engaged in paid labour in the labour force
#femaleh refers to the gender of household head- female led households are considered to be at greater risk of poverty

In [8]:
#satth refers to the highest level of education attained by household head these are:
#No education completed, primary school, junior secondary, senior secondary and higher education

#hhsize refers to the number of household members
#Quintile refers to the income quintile of household, compared to other households in the same region (there is significant inter-regional variation of incomes within Indonesia)

# Data cleaning and pre-processing

In [9]:
#Drop all observations for households in the treated group
household_data.drop(household_data[household_data["treated"]==1].index, inplace=True)

In [10]:
#Turn variable femaleh relating to female head of household into a dummy variable
household_data['femaleh'] = household_data['femaleh'].map({'Female': 1, 'Male': 0})

In [11]:
household_data.head()

,hhid,year,region,work10t12,work13t15,work16t18,uct,femaleh,satth,hhsize,...,m19t60,f61,m61,quint,rural,treatement,pretreatment,posttreatment,treated,ptreat
24,1.201160e+11,2005,Sumatra,0.0,0.0,0.0,0,0,Primary,10,...,0.200000,0.0,0.0,Quintile 1 (poorest),Rural,0.0,1.0,0.0,0.0,0.275653
25,1.201160e+11,2006,Sumatra,0.0,0.0,0.0,0,0,Primary,6,...,0.166667,0.0,0.0,Quintile 1 (poorest),Rural,0.0,0.0,1.0,0.0,0.336463
32,1.201160e+11,2005,Sumatra,0.0,0.0,0.0,0,0,Primary,7,...,0.142857,0.0,0.0,Quintile 1 (poorest),Rural,0.0,1.0,0.0,0.0,0.348078
33,1.201160e+11,2006,Sumatra,0.0,0.5,0.0,0,0,Primary,4,...,0.250000,0.0,0.0,Quintile 4,Rural,0.0,0.0,1.0,0.0,0.181095
34,1.201160e+11,2005,Sumatra,0.0,0.0,1.0,0,0,Primary,6,...,0.166667,0.0,0.0,Quintile 1 (poorest),Rural,0.0,1.0,0.0,0.0,0.448528


In [12]:
#We are now going to convert educational attainment of household head (satth) into into a series of dummy variables, to more accurately reflect the different bias in any logistic regression
#First, let's get a list of education types 
household_data.satth.unique()

array(['Primary', 'Junior secondary', 'Senior secondary',
       'No education completed', 'Higher'], dtype=object)

In [13]:
#based on the above we can see there are five different categories for education, which means we need four dummy variables
#creating five dummy variables would lead to a dummy variable trap and introduce a situation of perfect multi-collinearity, which is bad
#'No education completed' will be the default with no intercept
household_data['Primary'] = np.where(household_data['satth']=='Primary', 1, 0)
household_data['Junior secondary'] = np.where(household_data['satth']=='Junior secondary', 1, 0)
household_data['Senior secondary'] = np.where(household_data['satth']=='Senior secondary', 1, 0)
household_data['Higher'] = np.where(household_data['satth']=='Higher', 1, 0)



In [14]:
#Turn variable rural, which records whether a household rural or urban into a dummy variable
household_data['rural'] = household_data['rural'].map({'Rural': 1, 'Urban': 0})

In [15]:
#We are now going to convert income quintiles into into a series of dummy variables, to more accurately reflect the different bias in any logistic regression
#First, let's get a list of the income quintiles
household_data.quint.unique()

array(['Quintile 1 (poorest)', 'Quintile 4', 'Quintile 2',
       'Quintile 5 (richest)', 'Quintile 3'], dtype=object)

In [16]:
#Similar to education attainment, we will create n-1 dummy variables for income quintiles
household_data['Quintile 2'] = np.where(household_data['quint']=='Quintile 2', 1, 0)
household_data['Quintile 3'] = np.where(household_data['quint']=='Quintile 3', 1, 0)
household_data['Quintile 4'] = np.where(household_data['quint']=='Quintile 4', 1, 0)
household_data['Quintile 5'] = np.where(household_data['quint']=='Quintile 5 (richest)', 1, 0)

In [17]:
#In a similar fashion, we will now convert the various Indonesion provinces in variable Region into a series of dummy variables
household_data.region.unique()

array(['Sumatra', 'Java and Bali', 'Other islands', 'Kalimantan',
       'Sulawesi'], dtype=object)

In [18]:
#Create a series of n-1 dummy variables. 
#We will leave 'Other islands' as the default bias, with no specific dummy varaible of its own.
household_data['Sumatra'] = np.where(household_data['region']=='Sumatra', 1, 0)
household_data['Java and Bali'] = np.where(household_data['region']=='Java and Bali', 1, 0)
household_data['Kalimantan'] = np.where(household_data['region']=='Kalimantan', 1, 0)
household_data['Sulawesi'] = np.where(household_data['region']=='Sulawesi', 1, 0)

In [19]:
list(household_data.columns) 

['hhid',
 'year',
 'region',
 'work10t12',
 'work13t15',
 'work16t18',
 'uct',
 'femaleh',
 'satth',
 'hhsize',
 'f0t6',
 'm0t6',
 'f7t12',
 'm7t12',
 'f13t15',
 'f16t18',
 'f19t60',
 'm13t15',
 'm16t18',
 'm19t60',
 'f61',
 'm61',
 'quint',
 'rural',
 'treatement',
 'pretreatment',
 'posttreatment',
 'treated',
 'ptreat',
 'Primary',
 'Junior secondary',
 'Senior secondary',
 'Higher',
 'Quintile 2',
 'Quintile 3',
 'Quintile 4',
 'Quintile 5',
 'Sumatra',
 'Java and Bali',
 'Kalimantan',
 'Sulawesi']

In [20]:
#Drop unneccessary columns- either because we hae converted into multiple dummy variables, or because they are now redeundant
household_data=household_data.drop(['uct', 'satth', 'quint', 'treatement', 'pretreatment', 'treated', 'region'], axis = 1) 

In [21]:
#let's perform a quick check on 'work10t12' and see how many households are involved- potentially delete

In [22]:
seriesObj = household_data.apply(lambda x: True if x['work10t12'] > 0 else False , axis=1)
numOfRows = len(seriesObj[seriesObj == True].index)
print('Number of observations in dataframe in which children aged 10-12 are involved in workforce : ', numOfRows)

Number of observations in dataframe in which children aged 10-12 are involved in workforce :  16


In [23]:
#Decide to keep observations of 'work10t12'
#At the moment, the outcomes column 'work13t15' only records the proportion of an age group involved in labour force. 
#We want to convert this into a measure of proportion of total household.
household_data['%work10t12'] = household_data['work10t12']*(household_data['f7t12'] + household_data['m7t12'])

In [24]:
#Perform same routine on work13t15
household_data['%work13t15'] = household_data['work13t15']*(household_data['f13t15'] + household_data['m13t15'])

In [25]:
#Do the same for the 'work16t18' variable
household_data['%work16t18'] = household_data['work16t18']*(household_data['f16t18'] + household_data['m16t18'])

In [26]:
household_data['%work10t18'] = household_data['%work10t12']+household_data['%work16t18'] + household_data['%work13t15']

In [27]:
#Drop further unneccessary columns
household_data=household_data.drop(['work10t12', 'work13t15', 'work16t18'], axis = 1) 

In [28]:
household_data['Childlabour'] = np.where(household_data['%work10t18']>0, 1, 0)

In [29]:
household_data.head()

,hhid,year,femaleh,hhsize,f0t6,m0t6,f7t12,m7t12,f13t15,f16t18,...,Quintile 5,Sumatra,Java and Bali,Kalimantan,Sulawesi,%work10t12,%work13t15,%work16t18,%work10t18,Childlabour
24,1.201160e+11,2005,0,10,0.0,0.0,0.000000,0.000000,0.100000,0.100000,...,0,1,0,0,0,0.0,0.00,0.000000,0.000000,0
25,1.201160e+11,2006,0,6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0,1,0,0,0,0.0,0.00,0.000000,0.000000,0
32,1.201160e+11,2005,0,7,0.0,0.0,0.142857,0.000000,0.000000,0.142857,...,0,1,0,0,0,0.0,0.00,0.000000,0.000000,0
33,1.201160e+11,2006,0,4,0.0,0.0,0.000000,0.000000,0.500000,0.000000,...,0,1,0,0,0,0.0,0.25,0.000000,0.250000,1
34,1.201160e+11,2005,0,6,0.0,0.0,0.000000,0.333333,0.166667,0.166667,...,0,1,0,0,0,0.0,0.00,0.166667,0.166667,1


In [30]:
#Separate the observations from 2005 and 2006 into separate dataframes- 
#This will allow us to compare the same households from 2005 to 2006 and note any change in child labour participation
data_2005 = household_data[household_data.year == 2005]
data_2006 = household_data[household_data.year == 2006]

In [31]:
data_2005.head()

,hhid,year,femaleh,hhsize,f0t6,m0t6,f7t12,m7t12,f13t15,f16t18,...,Quintile 5,Sumatra,Java and Bali,Kalimantan,Sulawesi,%work10t12,%work13t15,%work16t18,%work10t18,Childlabour
24,1.201160e+11,2005,0,10,0.0,0.0,0.000000,0.000000,0.100000,0.100000,...,0,1,0,0,0,0.0,0.0,0.000000,0.000000,0
32,1.201160e+11,2005,0,7,0.0,0.0,0.142857,0.000000,0.000000,0.142857,...,0,1,0,0,0,0.0,0.0,0.000000,0.000000,0
34,1.201160e+11,2005,0,6,0.0,0.0,0.000000,0.333333,0.166667,0.166667,...,0,1,0,0,0,0.0,0.0,0.166667,0.166667,1
36,1.202050e+11,2005,0,5,0.0,0.0,0.000000,0.000000,0.000000,0.200000,...,0,1,0,0,0,0.0,0.0,0.200000,0.200000,1
38,1.202050e+11,2005,0,3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,1,1,0,0,0,0.0,0.0,0.000000,0.000000,0


In [32]:
#Let's merge the two dataframes, but we will only take the outcome columns from 2006
cleaned_data = pd.merge(data_2005,data_2006[['hhid','%work10t18', 'Childlabour']],on='hhid', how='left')

In [33]:
cleaned_data.shape

(4353, 38)

In [34]:
cleaned_data.head()

,hhid,year,femaleh,hhsize,f0t6,m0t6,f7t12,m7t12,f13t15,f16t18,...,Java and Bali,Kalimantan,Sulawesi,%work10t12,%work13t15,%work16t18,%work10t18_x,Childlabour_x,%work10t18_y,Childlabour_y
0,1.201160e+11,2005,0,10,0.0,0.0,0.000000,0.000000,0.100000,0.100000,...,0,0,0,0.0,0.0,0.000000,0.000000,0,0.00,0
1,1.201160e+11,2005,0,7,0.0,0.0,0.142857,0.000000,0.000000,0.142857,...,0,0,0,0.0,0.0,0.000000,0.000000,0,0.25,1
2,1.201160e+11,2005,0,6,0.0,0.0,0.000000,0.333333,0.166667,0.166667,...,0,0,0,0.0,0.0,0.166667,0.166667,1,0.00,0
3,1.202050e+11,2005,0,5,0.0,0.0,0.000000,0.000000,0.000000,0.200000,...,0,0,0,0.0,0.0,0.200000,0.200000,1,0.20,1
4,1.202050e+11,2005,0,3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0.0,0.0,0.000000,0.000000,0,0.00,0


In [35]:
# Now based on %work10t18_y and Childlabour_y, we have our final outcome variable, 
# which we can train our machine learning model to classify and predict. 
#At this point we might normally seek to apply some further scaling/normalisation or other transformation to our data. 
#However, our data is largely scaled to between 0 and 1 already.

# Let's export this as a csv for the next step
cleaned_data.to_csv('cleaned_data.csv') 